In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create two dfs for dropped_in and dropped_out
df_copy['count'] = 1
df_dropped_in = df_copy.copy()
df_dropped_out = df_copy.copy()
df_dropped_in.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,count
0,6328ea4e853c020ac7961848,1.224,883da106cbfffff,28.608149,77.379943,Delhi,883da106ddfffff,883da1a9a5fffff,28.612361,77.422325,28.609772,77.387451,034646,03,0345,Link,03:59,2,20220920,customerCancelled,customer not reachable,1
1,6328ed089aa7cf28bde99a64,9.273,883da116e1fffff,28.669567,77.226836,Delhi,883da116e1fffff,883da1b963fffff,28.716830,77.202640,28.668467,77.232117,035824,03,0345,Link,03:59,2,20220920,customerCancelled,Customer asked to cancel,1
2,6328e9be9aa7cf28bde99986,NaN,883da1075dfffff,28.546385,77.292213,Delhi,883da1075dfffff,883da10709fffff,28.538935,77.292213,28.546400,77.292267,034422,03,0330,Link,03:59,2,20220920,customerCancelled,order cancelled before rider accepted,1
3,6328e79c853c020ac7961785,0.401,883da13ae1fffff,28.489340,77.515602,Delhi,883da1385bfffff,883da13ae1fffff,28.489340,77.515602,28.478762,77.520325,033516,03,0330,Link,03:59,2,20220920,dropped,None,1
4,6328e850853c020ac79617c2,NaN,883da110abfffff,28.548103,77.125389,Delhi,883da110abfffff,883da116e1fffff,28.668766,77.230865,28.548162,77.125290,033816,03,0330,Link,03:59,2,20220920,customerCancelled,order cancelled before rider accepted,1


In [5]:
#Load the hexes to filter on
area1_hexes = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone1.csv')
area2_hexes = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone2.csv')

area1_hexes_list = list(area1_hexes['customer_location_hex_8'].unique())
area2_hexes_list = list(area2_hexes['customer_location_hex_8'].unique())

#Filter the dfs on the hexes identified for bangalore
df_dropped_in_area1 = df_dropped_in[df_dropped_in['drop_location_hex_8'].isin(area1_hexes_list)]
df_dropped_in_area1 = df_dropped_in_area1[df_dropped_in_area1['order_status'] == 'dropped']
df_dropped_in_area2 = df_dropped_in[df_dropped_in['drop_location_hex_8'].isin(area2_hexes_list)]
df_dropped_in_area2 = df_dropped_in_area2[df_dropped_in_area2['order_status'] == 'dropped']


df_dropped_out_area1 = df_dropped_out[df_dropped_out['customer_location_hex_8'].isin(area1_hexes_list)]
df_dropped_out_area1 = df_dropped_out_area1[df_dropped_out_area1['order_status'] == 'dropped']
df_dropped_out_area2 = df_dropped_out[df_dropped_out['customer_location_hex_8'].isin(area2_hexes_list)]
df_dropped_out_area2 = df_dropped_out_area2[df_dropped_out_area2['order_status'] == 'dropped']

In [6]:
#Groupby on date and hour to get the number of rides dropped_in/out
df_dropped_in_area1_rides = df_dropped_in_area1.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_in_area1_rides = df_dropped_in_area1_rides.rename(columns = {'count':'dropped_rides_in'})
df_dropped_in_area2_rides = df_dropped_in_area2.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_in_area2_rides = df_dropped_in_area2_rides.rename(columns = {'count':'dropped_rides_in'})

df_dropped_out_area1_rides = df_dropped_out_area1.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_out_area1_rides = df_dropped_out_area1_rides.rename(columns = {'count':'dropped_rides_out'})
df_dropped_out_area2_rides = df_dropped_out_area2.groupby(['yyyymmdd', 'hour'])['count'].sum().reset_index()
df_dropped_out_area2_rides = df_dropped_out_area2_rides.rename(columns = {'count':'dropped_rides_out'})

In [7]:
#Merge dropped_in and dropped_out
df_replenishment_area1 = df_dropped_in_area1_rides.merge(df_dropped_out_area1_rides, on = ['yyyymmdd', 'hour'], how = 'left')
df_replenishment_area2 = df_dropped_in_area2_rides.merge(df_dropped_out_area2_rides, on = ['yyyymmdd', 'hour'], how = 'left')

df_replenishment_area1 = df_replenishment_area1.sort_values(by = ['yyyymmdd', 'hour'], ascending=True)
df_replenishment_area2 = df_replenishment_area2.sort_values(by = ['yyyymmdd', 'hour'], ascending=True)

#Calculate replenishment
df_replenishment_area1['replenishment'] = df_replenishment_area1['dropped_rides_in'] - df_replenishment_area1['dropped_rides_out']
df_replenishment_area2['replenishment'] = df_replenishment_area2['dropped_rides_in'] - df_replenishment_area2['dropped_rides_out']
df_replenishment_area1.head()

,yyyymmdd,hour,dropped_rides_in,dropped_rides_out,replenishment
0,20220919,00,69,89,-20
1,20220919,01,46,47,-1
2,20220919,02,36,40,-4
3,20220919,03,19,24,-5
4,20220919,04,13,22,-9


In [8]:
#Save the replenishment to excel
df_replenishment_area1.to_csv('replenishment_delhi_area1.csv', index = False)
df_replenishment_area2.to_csv('replenishment_delhi_area2.csv', index = False)